In [1]:
import pandas as pd
import numpy as np
from concurrent.futures import ProcessPoolExecutor
from itertools import combinations

In [2]:
df_movie = pd.read_pickle('data/df_movie.pkl')
df_movie.head()

,Movie_name,Movie_release,Movie_revenue,Movie_runtime,Movie_languages,Movie_countries,Main_genre,Sec_Genre,Movie_rating,Producer,...,Actor_ethnicity,Actor_name,Actor_age_release,Inflation Factor for 2023,2023 valued revenue,Avg_revenue_per_film_at_release,Longevity,Number_of_film_at_release,Avg_rating_per_film_at_release,First_film
403088,The Fox and the Hound,1981.0,63456988.0,83.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}",Adventure,Children's/Family,NaN,NaN,...,NaN,'Squeeks' the Caterpillar,NaN,3.38,2.144846e+08,6.345699e+07,0.0,1,NaN,True
400285,Miss March,2009.0,4591629.0,90.0,"{""/m/05zjd"": ""Portuguese Language"", ""/m/02h40l...","{""/m/09c7w0"": ""United States of America""}",Road movie,Sex comedy,NaN,NaN,...,NaN,40 Glocc,29.0,1.43,6.566029e+06,4.591629e+06,0.0,1,NaN,True
71882,Get Rich or Die Tryin',2005.0,46442528.0,117.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}",Crime Fiction,Hip hop movies,5.4,Jimmy Iovine,...,/m/0x67,50 Cent,30.0,1.58,7.337919e+07,4.644253e+07,0.0,1,5.400000,True
419847,Home of the Brave,2006.0,499620.0,106.0,"{""/m/0jzc"": ""Arabic Language"", ""/m/02h40lc"": ""...","{""/m/09c7w0"": ""United States of America"", ""/m/...",Drama,War film,5.6,NaN,...,/m/0x67,50 Cent,31.0,1.53,7.644186e+05,2.347107e+07,1.0,2,5.500000,False
126916,Righteous Kill,2008.0,76747202.0,100.0,"{""/m/06b_j"": ""Russian Language"", ""/m/02h40lc"":...","{""/m/09c7w0"": ""United States of America""}",Thriller,Crime Fiction,6.0,NaN,...,/m/0x67,50 Cent,33.0,1.43,1.097485e+08,4.122978e+07,3.0,3,5.666667,False


In [3]:
df_movie = df_movie[['Movie_name', 'Movie_release', 'Actor_name', 'Actor_age_release', 'Number_of_film_at_release', 'Avg_revenue_per_film_at_release', 'First_film']]
df_movie.head()

,Movie_name,Movie_release,Actor_name,Actor_age_release,Number_of_film_at_release,Avg_revenue_per_film_at_release,First_film
403088,The Fox and the Hound,1981.0,'Squeeks' the Caterpillar,NaN,1,6.345699e+07,True
400285,Miss March,2009.0,40 Glocc,29.0,1,4.591629e+06,True
71882,Get Rich or Die Tryin',2005.0,50 Cent,30.0,1,4.644253e+07,True
419847,Home of the Brave,2006.0,50 Cent,31.0,2,2.347107e+07,False
126916,Righteous Kill,2008.0,50 Cent,33.0,3,4.122978e+07,False


In [4]:
# Group by 'Movie_name' and aggregate 'Actor_name' into lists
actor_pairs = df_movie.groupby(['Movie_name', 'Movie_release'])['Actor_name'].apply(list)

# For each movie, create all possible pairs of actors without duplication
actor_pairs = actor_pairs.apply(lambda x: list(combinations(sorted(set(x)), 2)))

df_pairs = pd.DataFrame(actor_pairs)
df_pairs = df_pairs.reset_index()
df_pairs = df_pairs.rename(columns={'Actor_name': 'Actor_pairs'})
df_pairs = df_pairs.explode('Actor_pairs')
df_pairs.head()

,Movie_name,Movie_release,Actor_pairs
0,'Til There Was You,1997.0,"(Alice Drummond, Christine Ebersole)"
0,'Til There Was You,1997.0,"(Alice Drummond, Craig Bierko)"
0,'Til There Was You,1997.0,"(Alice Drummond, Dylan McDermott)"
0,'Til There Was You,1997.0,"(Alice Drummond, Jeanne Tripplehorn)"
0,'Til There Was You,1997.0,"(Alice Drummond, Jennifer Aniston)"


In [5]:
df_all = pd.read_pickle('data/df_all.pkl')
df_all.head()

,Movie_name,Movie_release,Movie_revenue,Movie_runtime,Movie_languages,Movie_countries,Main_genre,Sec_Genre,Movie_rating,Producer,...,Director,Writer,Freebase_ID,Char_name,Actor_birth,Actor_gender,Actor_height,Actor_ethnicity,Actor_name,Actor_age_release
0,Ghosts of Mars,2001.0,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}",Thriller,Science Fiction,4.9,Sandy King,...,John Carpenter,NaN,/m/03vyhn,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0
1,Ghosts of Mars,2001.0,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}",Thriller,Science Fiction,4.9,Sandy King,...,John Carpenter,NaN,/m/03vyhn,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0
2,Ghosts of Mars,2001.0,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}",Thriller,Science Fiction,4.9,Sandy King,...,John Carpenter,NaN,/m/03vyhn,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0
3,Ghosts of Mars,2001.0,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}",Thriller,Science Fiction,4.9,Sandy King,...,John Carpenter,NaN,/m/03vyhn,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0
4,Ghosts of Mars,2001.0,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}",Thriller,Science Fiction,4.9,Sandy King,...,John Carpenter,NaN,/m/03vyhn,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0


In [6]:
df_test = pd.read_csv('data/df_movie.csv')
df_test.rename(columns={'Movie_release_x': 'Movie_release'}, inplace=True)
df_test.head()

,Movie_name,Movie_release,Movie_revenue,Movie_runtime,Movie_countries,Genre_dict,Main_genre,Sec_Genre,imdb_rating,runtime_min,...,Actor_gender,Actor_height,Actor_ethnicity,Actor_name,Actor_age_release,Avg_revenue_per_film_at_release,Longevity,Number_of_film_at_release,Avg_rating_per_film_at_release,First_film
0,The Fox and the Hound,1981.0,63456988.0,83.0,"{""/m/09c7w0"": ""United States of America""}","{'/m/03k9fj': 'Adventure', '/m/0hj3myq': ""Chil...",Adventure,Children's/Family,NaN,NaN,...,NaN,NaN,NaN,'Squeeks' the Caterpillar,NaN,6.345699e+07,0.0,1,NaN,True
1,Miss March,2009.0,4591629.0,90.0,"{""/m/09c7w0"": ""United States of America""}","{'/m/04228s': 'Road movie', '/m/0gsy3b': 'Sex ...",Road movie,Sex comedy,NaN,NaN,...,M,NaN,NaN,40 Glocc,29.0,4.591629e+06,0.0,1,NaN,True
2,Get Rich or Die Tryin',2005.0,46442528.0,117.0,"{""/m/09c7w0"": ""United States of America""}","{'/m/0lsxr': 'Crime Fiction', '/m/026v1nw': 'H...",Crime Fiction,Hip hop movies,5.4,117.0,...,M,1.829,/m/0x67,50 Cent,30.0,4.644253e+07,0.0,1,5.400000,True
3,Home of the Brave,2006.0,499620.0,106.0,"{""/m/09c7w0"": ""United States of America"", ""/m/...","{'/m/07s9rl0': 'Drama', '/m/082gq': 'War film'}",Drama,War film,5.6,106.0,...,M,1.829,/m/0x67,50 Cent,31.0,2.347107e+07,1.0,2,5.500000,False
4,Righteous Kill,2008.0,76747202.0,100.0,"{""/m/09c7w0"": ""United States of America""}","{'/m/01jfsb': 'Thriller', '/m/0lsxr': 'Crime F...",Thriller,Crime Fiction,6.0,101.0,...,M,1.829,/m/0x67,50 Cent,33.0,4.122978e+07,3.0,3,5.666667,False


In [7]:
df_pairs = df_pairs.merge(df_all[['Movie_name', 'Movie_release', 'Movie_revenue', 'Movie_rating']], on=['Movie_name', 'Movie_release'], how='left')

In [8]:
df_pairs.dropna(subset=['Actor_pairs', 'Movie_release', 'Movie_revenue', 'Movie_rating'], inplace=True)

In [9]:
df_pairs['Actor1'] = df_pairs['Actor_pairs'].apply(lambda x: x[0])
df_pairs['Actor2'] = df_pairs['Actor_pairs'].apply(lambda x: x[1])
df_pairs.reset_index(drop=True, inplace=True)

In [10]:
def query_info_for_actor_pairs(row, df, infos):
    actor1 = row['Actor1']
    actor2 = row['Actor2']
    movie_name = row['Movie_name']
    movie_release = row['Movie_release']

    # Create a mask for the condition
    condition_mask = (df['Movie_name'] == movie_name) & (df['Movie_release'] == movie_release)

    # Filter the DataFrame based on the condition
    filtered_df = df[condition_mask]

    # Extract the relevant information for actor1 and actor2
    info1 = filtered_df.loc[filtered_df['Actor_name'] == actor1, infos].values[0]
    info2 = filtered_df.loc[filtered_df['Actor_name'] == actor2, infos].values[0]

    return info1, info2

In [11]:
def all_infos(row):
    infos1, infos2 = query_info_for_actor_pairs(row, df_movie, ['Actor_age_release', 'Number_of_film_at_release', 'Avg_revenue_per_film_at_release', 'First_film'])

    age_difference = np.abs(infos1[0] - infos2[0])
    film_count_difference = np.abs(infos1[1] - infos2[1])
    average_revenue_difference = np.abs(infos1[2] - infos2[2])
    first_film = (infos1[3] and infos2[3])
    first_film_for_one = (infos1[3] or infos2[3])

    print(f"Progress: {(row.name/len(df_pairs)):.2%}")

    return age_difference, film_count_difference, average_revenue_difference, first_film, first_film_for_one

In [12]:
def number_of_films_together(row):
    df = df_movie
    actor1 = row['Actor1']
    actor2 = row['Actor2']
    movie_release = row['Movie_release']

    condition_mask1 = ((df['Movie_release'] < movie_release) & (df['Actor_name'] == actor1))
    condition_mask2 = ((df['Movie_release'] < movie_release) & (df['Actor_name'] == actor2))

    movie_name1 = df.loc[condition_mask1, 'Movie_name'].values.tolist()
    movie_name2 = df.loc[condition_mask2, 'Movie_name'].values.tolist()
    
    print(f"Progress: {(row.name/len(df_pairs)):.2%}")

    return len(set(movie_name1).intersection(set(movie_name2)))


In [ ]:
infos = df_pairs.apply(lambda x: all_infos(x), axis=1)

In [ ]:
df_pairs[['Age_difference', 'Film_count_difference', 'Average_revenue_difference', 'First_film', 'First_film_for_one']] = pd.DataFrame(infos.tolist(), index=df_pairs.index)

In [ ]:
films_together = df_pairs.apply(lambda x: number_of_films_together(x, df_movie), axis=1)

In [ ]:
df_pairs['Number_of_films_together'] = films_together